<div style="text-align: right"> Ali Emre Öz </div>
<div style="text-align: right"> 213950785 </div>

<h3 align="center">EECS 461 - MACHINE LEARNING PROJECT FINAL REPORT</h3> 

In my project, I will try to predict the genre of the movie using data about movie; like budget, revenue, release year, imdb rating, used word in title etc.

For this purpose, I found a two different dataset from two different source. One from IMDB and other from TMDB. At first I try to explore this dataset and clean a little bit. Then I try to visualize some information about this cleaned dataset. After that I will try to set my data ready for machine learning algorithm by doing some scaling,one-hot-encoding and bag of word analysis. And then I split my data for training and testing. As my main goal is try to predict movie genre from some genres. This is a multiclass classification task. I will use Random Forest Classifier, Logistic Regression and Ada Boost Classifier with one vs all approach. After that, I will try to implement dimensionality reduction methods by applying PCA. For performance measurement, I will use Hamming Loss.

I split my notebook as 4 different purpose.
* Data Exploration and Cleanup
* Data Visualization
* Data Preprocessing
* Classification Algorithms and Performance Measurements

#### Data Exploration and Cleanup

In [1]:
%matplotlib inline
%pylab inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
imdb_data_1 = pd.read_csv("imdb1.csv")
imdb_data_2 = pd.read_csv("imdb2.csv")
cols = imdb_data_1.columns
imdb_data_3 = pd.read_table("imdb3.txt", sep='\t', header=None, names = cols[1:])
del imdb_data_1['Unnamed: 0']
del imdb_data_2['Unnamed: 0']
print imdb_data_1.shape
print imdb_data_2.shape
print imdb_data_3.shape

(25745, 45)
(324, 45)
(40557, 45)


In [3]:
frames = [imdb_data_1, imdb_data_2, imdb_data_3]
imdb_data = pd.concat(frames)
print imdb_data.shape

(66626, 45)


In [4]:
imdb_data.columns

Index([u'id', u'title', u'canonical title', u'imdbIndex', u'kind', u'year',
       u'rating', u'mpaa', u'votes', u'runtimes', u'color info', u'genres',
       u'languages', u'plot', u'countries', u'director', u'writer', u'cast',
       u'certificates', u'Action', u'Adventure', u'Adult', u'Animation',
       u'Comedy', u'Crime', u'Documentary', u'Drama', u'Family', u'Fantasy',
       u'Film-Noir', u'Horror', u'Musical', u'Mystery', u'Romance', u'Sci-Fi',
       u'Short', u'Thriller', u'War', u'Western', u'Talk-Show', u'News',
       u'Game-Show', u'Reality-TV', u'History', u'Sport'],
      dtype='object')

In [5]:
tmdb_data = pd.read_csv("tmdb.csv")
print tmdb_data.shape

(79716, 16)


In [6]:
tmdb_data.columns

Index([u'movie_id', u'revenue', u'overview', u'genres', u'title',
       u'vote_count', u'imdb_id', u'popularity', u'budget', u'vote_average',
       u'runtime', u'red_pixel', u'green_pixel', u'blue_pixel',
       u'release_date_month', u'release_date_year'],
      dtype='object')

##### Delete the unnecessary features from both dataset

In [7]:
for column_name in ["votes","imdbIndex","canonical title","mpaa","kind","runtimes","plot","certificates","color info","countries","languages"]:
    del imdb_data[column_name]

In [8]:
for column_name in ["release_date_year","release_date_month","blue_pixel","green_pixel","red_pixel","title","genres","vote_count"]:
    del tmdb_data[column_name]

In [9]:
result = tmdb_data.merge(imdb_data, left_on='imdb_id', right_on='id', how='inner')

In [10]:
print result.shape
print result.columns
result.head()

(63911, 42)
Index([u'movie_id', u'revenue', u'overview', u'imdb_id', u'popularity',
       u'budget', u'vote_average', u'runtime', u'id', u'title', u'year',
       u'rating', u'genres', u'director', u'writer', u'cast', u'Action',
       u'Adventure', u'Adult', u'Animation', u'Comedy', u'Crime',
       u'Documentary', u'Drama', u'Family', u'Fantasy', u'Film-Noir',
       u'Horror', u'Musical', u'Mystery', u'Romance', u'Sci-Fi', u'Short',
       u'Thriller', u'War', u'Western', u'Talk-Show', u'News', u'Game-Show',
       u'Reality-TV', u'History', u'Sport'],
      dtype='object')


,movie_id,revenue,overview,imdb_id,popularity,budget,vote_average,runtime,id,title,...,Short,Thriller,War,Western,Talk-Show,News,Game-Show,Reality-TV,History,Sport
0,106505,0,"Relentlessly gruesome, South Korean filmmaker ...",tt2349554,0.000814,0,5.8,75.0,tt2349554,Hier bei mir,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,106507,0,"takes pride in calling himself a ""Sesky Video ...",tt2347289,0.003110,0,0.0,145.0,tt2347289,The Jubilee Girl,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,106509,0,A young millionaire (Richard Carlson) joins th...,tt0034853,0.000155,0,6.0,65.0,tt0034853,Highways by Night,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,106511,0,"Rd Cokro, a wealthy businessman, obtains his w...",tt0285808,0.000083,0,0.0,82.0,tt0285808,Nyi blorong,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,106513,0,"The city's District Attorney is murdered, and ...",tt0036255,0.010632,0,6.5,74.0,tt0036255,The Pay Off,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Replace Nan

In [11]:
result = result.replace(to_replace="null", value=np.nan, inplace=False)

In [12]:
print result.shape
result.head()

(63911, 42)


,movie_id,revenue,overview,imdb_id,popularity,budget,vote_average,runtime,id,title,...,Short,Thriller,War,Western,Talk-Show,News,Game-Show,Reality-TV,History,Sport
0,106505,0,"Relentlessly gruesome, South Korean filmmaker ...",tt2349554,0.000814,0,5.8,75.0,tt2349554,Hier bei mir,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,106507,0,"takes pride in calling himself a ""Sesky Video ...",tt2347289,0.003110,0,0.0,145.0,tt2347289,The Jubilee Girl,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,106509,0,A young millionaire (Richard Carlson) joins th...,tt0034853,0.000155,0,6.0,65.0,tt0034853,Highways by Night,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,106511,0,"Rd Cokro, a wealthy businessman, obtains his w...",tt0285808,0.000083,0,0.0,82.0,tt0285808,Nyi blorong,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,106513,0,"The city's District Attorney is murdered, and ...",tt0036255,0.010632,0,6.5,74.0,tt0036255,The Pay Off,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
del result["id"]

In [14]:
result.columns

Index([u'movie_id', u'revenue', u'overview', u'imdb_id', u'popularity',
       u'budget', u'vote_average', u'runtime', u'title', u'year', u'rating',
       u'genres', u'director', u'writer', u'cast', u'Action', u'Adventure',
       u'Adult', u'Animation', u'Comedy', u'Crime', u'Documentary', u'Drama',
       u'Family', u'Fantasy', u'Film-Noir', u'Horror', u'Musical', u'Mystery',
       u'Romance', u'Sci-Fi', u'Short', u'Thriller', u'War', u'Western',
       u'Talk-Show', u'News', u'Game-Show', u'Reality-TV', u'History',
       u'Sport'],
      dtype='object')

##### Delete some label which does not make sense as others

In [15]:
cols = result.columns
cols = cols.map(lambda x: x.replace('-', '_'))
result.columns = cols
result = result[result.Sport == 0]
result = result[result.Adult == 0]
result = result[result.Film_Noir == 0]
result = result[result.Short == 0]
result = result[result.Talk_Show == 0]
result = result[result.News == 0]
result = result[result.Game_Show == 0]
result = result[result.Reality_TV == 0]
result.shape

(47673, 41)

In [16]:
del result['Sport']
del result['Adult']
del result['Film_Noir']
del result['Short']
del result['Talk_Show']
del result['News']
del result['Game_Show']
del result['Reality_TV']
result.shape

(47673, 33)

##### Add forgetten label, Biography

In [17]:
result['Biography'] = 0
result.shape

(47673, 34)

In [18]:
for i in result["genres"]:
    if "Biography" in i:
        result.loc[result.genres == i, "Biography"] = 1        

In [19]:
example_movie = result.iloc[0]
example_movie

movie_id                                                   106507
revenue                                                         0
overview        takes pride in calling himself a "Sesky Video ...
imdb_id                                                 tt2347289
popularity                                                0.00311
budget                                                          0
vote_average                                                    0
runtime                                                       145
title                                            The Jubilee Girl
year                                                         2002
rating                                                        6.8
genres                                             [u'Biography']
director                                                      NaN
writer                                                        NaN
cast            ['Bryan Adams', 'Roberto Alagna', 'Thomas Alle...
Action    

##### Delete person-based feature, because it made the job harder even it helped to make a better prediction.

In [20]:
del result['director']
del result['writer']
del result['cast']
del result['genres']

In [21]:
result.columns

Index([u'movie_id', u'revenue', u'overview', u'imdb_id', u'popularity',
       u'budget', u'vote_average', u'runtime', u'title', u'year', u'rating',
       u'Action', u'Adventure', u'Animation', u'Comedy', u'Crime',
       u'Documentary', u'Drama', u'Family', u'Fantasy', u'Horror', u'Musical',
       u'Mystery', u'Romance', u'Sci_Fi', u'Thriller', u'War', u'Western',
       u'History', u'Biography'],
      dtype='object')

In [22]:
example_movie = result.iloc[0]
example_movie

movie_id                                                   106507
revenue                                                         0
overview        takes pride in calling himself a "Sesky Video ...
imdb_id                                                 tt2347289
popularity                                                0.00311
budget                                                          0
vote_average                                                    0
runtime                                                       145
title                                            The Jubilee Girl
year                                                         2002
rating                                                        6.8
Action                                                          0
Adventure                                                       0
Animation                                                       0
Comedy                                                          0
Crime     

##### Save the result for visualization and preprocessing

In [24]:
result.to_csv('result.csv')